In [1]:
!pip install albumentations opencv-python

In [2]:
import os
from pathlib import Path
import cv2
import random
from tqdm import tqdm
import albumentations as A

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from pathlib import Path

# === Original data paths ===
original_root = Path("/content/drive/MyDrive/Projects/Car Detection v2/Multi Class Data")

original_train_images = original_root / "train/images"
original_train_labels = original_root / "train/labels"

original_valid_images = original_root / "valid/images"
original_valid_labels = original_root / "valid/labels"

# === Augmented data output paths ===
augmented_root = Path("/content/drive/MyDrive/Projects/Car Detection v2/Multi Class Data - Augmented")

augmented_train_images = augmented_root / "train/images"
augmented_train_labels = augmented_root / "train/labels"

augmented_valid_images = augmented_root / "valid/images"
augmented_valid_labels = augmented_root / "valid/labels"

# Create output directories if not exist
for path in [augmented_train_images, augmented_train_labels, augmented_valid_images, augmented_valid_labels]:
    path.mkdir(parents=True, exist_ok=True)


In [5]:
# CLASS DISTRIBUTION (optional imbalance handling)
from collections import defaultdict

class_distribution = defaultdict(int)
for label_file in original_train_labels.glob('*.txt'):
    with open(label_file, 'r') as f:
        for line in f.readlines():
            class_id = line.strip().split()[0]
            class_distribution[class_id] += 1

print("Class Distribution:", dict(class_distribution))

Class Distribution: {'3': 459, '12': 360, '7': 2228, '2': 113, '10': 666, '8': 93, '5': 104, '9': 222, '4': 32, '0': 64, '1': 48, '11': 31, '6': 36}


In [6]:
MIN_SAMPLES = max(class_distribution.values())
oversample_map = {
    class_id: (MIN_SAMPLES - count)
    for class_id, count in class_distribution.items()
    if count < MIN_SAMPLES
}

In [7]:
# AUGMENTATION PIPELINE
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.4),
    A.HueSaturationValue(p=0.3),
    A.MotionBlur(blur_limit=3, p=0.2),
    A.RandomScale(scale_limit=0.2, p=0.3),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, p=0.5),
    A.Resize(640, 640),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [8]:
# FUNCTION TO READ YOLO LABEL FILE
def read_yolo_label(path):
    boxes = []
    class_labels = []
    with open(path, 'r') as f:
        for line in f.readlines():
            values = list(map(float, line.strip().split()))
            class_labels.append(int(values[0]))
            boxes.append(values[1:5])  # cx, cy, w, h
    return boxes, class_labels

In [ ]:
# APPLY AUGMENTATION & SAVE
counter = 0
for label_file in tqdm(list(original_train_labels.glob('*.txt'))):
    img_file = original_train_images / (label_file.stem + '.jpg')
    image = cv2.imread(str(img_file))
    if image is None:
        continue

    boxes, class_labels = read_yolo_label(label_file)
    if not boxes:
        continue

    # base copy
    for _ in range(3):  # number of augmentations per image
        augmented = transform(image=image, bboxes=boxes, class_labels=class_labels)
        aug_img = augmented['image']
        aug_boxes = augmented['bboxes']
        aug_classes = augmented['class_labels']

        # Save image
        filename = f"{label_file.stem}_aug{counter}"
        out_img_path = augmented_train_images / f"{filename}.jpg"
        out_lbl_path = augmented_train_labels / f"{filename}.txt"

        cv2.imwrite(str(out_img_path), aug_img)

        # Save label
        with open(out_lbl_path, 'w') as f:
            for cls, box in zip(aug_classes, aug_boxes):
                f.write(f"{cls} {' '.join(map(str, box))}\n")

        counter += 1

print(f"Augmentation complete. {counter} images saved to {augmented_train_images}")

100%|██████████| 1896/1896 [10:59<00:00,  2.87it/s]

Augmentation complete. 5682 images saved to /content/drive/MyDrive/Projects/Car Detection v2/Multi Class Data - Augmented/train/images


In [9]:
# APPLY AUGMENTATION & SAVE - Data Validation
counter = 0
for label_file in tqdm(list(original_valid_labels.glob('*.txt'))):
    img_file = original_valid_images / (label_file.stem + '.jpg')
    image = cv2.imread(str(img_file))
    if image is None:
        continue

    boxes, class_labels = read_yolo_label(label_file)
    if not boxes:
        continue

    # base copy
    for _ in range(3):  # number of augmentations per image
        augmented = transform(image=image, bboxes=boxes, class_labels=class_labels)
        aug_img = augmented['image']
        aug_boxes = augmented['bboxes']
        aug_classes = augmented['class_labels']

        # Save image
        filename = f"{label_file.stem}_aug{counter}"
        out_img_path = augmented_valid_images / f"{filename}.jpg"
        out_lbl_path = augmented_valid_labels / f"{filename}.txt"

        cv2.imwrite(str(out_img_path), aug_img)

        # Save label
        with open(out_lbl_path, 'w') as f:
            for cls, box in zip(aug_classes, aug_boxes):
                f.write(f"{cls} {' '.join(map(str, box))}\n")

        counter += 1

print(f"Augmentation complete. {counter} images saved to {augmented_valid_images}")

100%|██████████| 312/312 [02:43<00:00,  1.91it/s]

Augmentation complete. 936 images saved to /content/drive/MyDrive/Projects/Car Detection v2/Multi Class Data - Augmented/valid/images
